In [1]:
import tensorflow as tf
import numpy as np
from pathlib import Path

2023-10-30 16:10:08.099044: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
DATA_PATH = Path("/scratch/ajb5d/ecg/tfrecords/")
TRAIN_RECS = list(DATA_PATH.glob("train*.tfrecords"))
VAL_RECS = list(DATA_PATH.glob("val*.tfrecords"))

In [3]:
BATCH_SIZE = 64

record_format = {
    'ecg/data': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'age': tf.io.FixedLenFeature([], tf.float32),
    'potassium': tf.io.FixedLenFeature([], tf.float32),
    'gender': tf.io.FixedLenFeature([], tf.int64),
}

def _parse_record(record):
    example = tf.io.parse_single_example(record, record_format)
    ecg_data = tf.reshape(example['ecg/data'], [5000,12])
    label = example['potassium']
    return ecg_data, label

def drop_na_ages(x,y):
    return not tf.math.reduce_any(tf.math.is_nan(y))

def load_dataset(filenames):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(_parse_record, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.filter(drop_na_ages)
    return dataset

def get_dataset(filenames, labeled=True):
    dataset = load_dataset(filenames)
    dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

In [4]:
train_dataset = get_dataset(TRAIN_RECS)
val_dataset = get_dataset(VAL_RECS)

2023-10-30 16:10:12.998035: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1636] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78791 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:bd:00.0, compute capability: 8.0


In [5]:
# tf.keras.mixed_precision.set_global_policy('mixed_float16')

In [6]:
def conv_block(x, n, k, mp, groups = 12):
    x = tf.keras.layers.Conv1D(n * groups, k, padding='same', groups = groups)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    if mp:
        x = tf.keras.layers.MaxPooling1D(mp)(x)

    return x

input_layer = tf.keras.layers.Input(shape=(5000, 12))
x = input_layer

x = conv_block(x, 16, 7, 2)
x = conv_block(x, 16, 5, 4)
x = conv_block(x, 32, 5, 2)
x = conv_block(x, 32, 5, 4)
x = conv_block(x, 64, 5, 2)
x = conv_block(x, 64, 3, 2)
x = conv_block(x, 64, 3, 2)
x = conv_block(x, 64, 3, 2)

x = tf.keras.layers.Conv1D(128, 12, padding = 'same', data_format = 'channels_first')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.MaxPooling1D()(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(64)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1)(x)

model = tf.keras.models.Model(input_layer, x)

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=['mse', 'mae']
)

In [7]:
from datetime import datetime
import os

def make_checkpoint_dir(data_path, label):
    current_datetime = datetime.now()
    formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")
    output_dir = f"{label}-{formatted_datetime}"
    output_path = f"{data_path}/{output_dir}"
    
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    return output_path

In [8]:
callbacks = [
    tf.keras.callbacks.TerminateOnNaN(),
    tf.keras.callbacks.ReduceLROnPlateau(),
    tf.keras.callbacks.ModelCheckpoint(make_checkpoint_dir("data/models", "cnn-potassium"))
]

model.fit(train_dataset, epochs=10, validation_data=val_dataset, callbacks=callbacks)

Epoch 1/10


2023-10-30 16:10:18.289705: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55aeb72fe3b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-30 16:10:18.290099: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-SXM4-80GB, Compute Capability 8.0
2023-10-30 16:10:18.490850: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8904
2023-10-30 16:10:20.136095: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2023-10-30 16:10:22.103326: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:625] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-10-30 16:10:22.358460: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to en

   4248/Unknown - 357s 80ms/step - loss: 1.2119 - mse: 1.2119 - mae: 0.8056

2023-10-30 16:16:11.157714: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 16263120092429597367
2023-10-30 16:16:11.157790: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 6257262231560064895


INFO:tensorflow:Assets written to: data/models/cnn-potassium-2023-10-30_16-10-14/assets


...

4248/4248 [==============================] - 359s 84ms/step - loss: 0.4840 - mse: 0.4840 - mae: 0.4782 - val_loss: 0.4856 - val_mse: 0.4856 - val_mae: 0.4742 - lr: 0.0010
Epoch 9/10
4248/4248 [==============================] - ETA: 0s - loss: 0.4813 - mse: 0.4813 - mae: 0.4768INFO:tensorflow:Assets written to: data/models/cnn-potassium-2023-10-30_16-10-14/assets


INFO:tensorflow:Assets written to: data/models/cnn-potassium-2023-10-30_16-10-14/assets


4248/4248 [==============================] - 359s 84ms/step - loss: 0.4813 - mse: 0.4813 - mae: 0.4768 - val_loss: 0.5102 - val_mse: 0.5102 - val_mae: 0.4714 - lr: 0.0010
Epoch 10/10
4248/4248 [==============================] - ETA: 0s - loss: 0.4766 - mse: 0.4766 - mae: 0.4749INFO:tensorflow:Assets written to: data/models/cnn-potassium-2023-10-30_16-10-14/assets


INFO:tensorflow:Assets written to: data/models/cnn-potassium-2023-10-30_16-10-14/assets


4248/4248 [==============================] - 366s 86ms/step - loss: 0.4766 - mse: 0.4766 - mae: 0.4749 - val_loss: 0.5130 - val_mse: 0.5130 - val_mae: 0.4715 - lr: 0.0010


In [9]:
model.save("data/models/cnn-potassium.keras")